## *Setup required for Playground*

The online playground requires a little bit of code to run first.

In [ ]:
import piplite
await piplite.install('kloppy')
await piplite.install('pandas')

# Code data

Apart from event- and tracking data analytics often use code data. This type of data can be collected by hand using tools like SportsCode. 

Kloppy allows easy read AND write functionallity for the codes in SportsCode XML format. 

## Reading XML file

In [ ]:
from kloppy import sportscode

In [ ]:
with open("file.xml", "w") as fp:
    fp.write("""<?xml version="1.0"?>
<file>
    <ALL_INSTANCES>
        <instance>
            <ID>P1</ID>
            <start>3.6</start>
            <end>9.7</end>
            <code>PASS</code>
            <label>
                <group>Team</group>
                <text>Henkie</text>
            </label>
            <label>
                <group>Packing.Value</group>
                <text>1</text>
            </label>
            <label>
                <group>Receiver</group>
                <text>Klaas N&#xF8;me</text>
            </label>
        </instance>
        <instance>
            <ID>P2</ID>
            <start>68.3</start>
            <end>74.5</end>
            <code>PASS</code>
            <label>
                <group>Team</group>
                <text>Henkie</text>
            </label>
            <label>
                <group>Packing.Value</group>
                <text>3</text>
            </label>
            <label>
                <group>Receiver</group>
                <text>Piet</text>
            </label>
        </instance>
        <instance>
            <ID>P3</ID>
            <start>103.6</start>
            <end>109.6</end>
            <code>SHOT</code>
            <label>
                <group>Team</group>
                <text>Henkie</text>
            </label>
            <label>
                <group>Expected.Goal.Value</group>
                <text>0.13</text>
            </label>
        </instance>
    </ALL_INSTANCES>
</file>""")
    
code_dataset = sportscode.load("file.xml")


In [ ]:
code_dataset.to_pandas()

The code dataset also allows filtering the codes

In [ ]:
passes = code_dataset.filter(lambda code: code.code == 'PASS')
passes.to_pandas()

## Writing XML file

In [ ]:
sportscode.save(passes, "file.xml")

with open("file.xml", "r") as fp:
    print(fp.read())

## Converting event dataset into XML dataset

In [ ]:
from kloppy import statsbomb

dataset = statsbomb.load_open_data()

In [ ]:
from kloppy.domain import Code, CodeDataset, EventType

dataset_shots = dataset.filter(
    lambda event: event.event_type == EventType.SHOT
)

code_dataset = (
    CodeDataset
    .from_dataset(
        dataset_shots,
        lambda event: Code(
            code_id=None,  # make it auto increment on write
            code=event.event_name,
            period=event.period,
            timestamp=max(0, event.timestamp - 7),
            end_timestamp=event.timestamp + 5,
            labels={
                'Player': str(event.player),
                'Team': str(event.team)
            },
            
            # In the future next two won't be needed anymore
            ball_owning_team=None,
            ball_state=None
        )
    )
)

In [ ]:
code_dataset.to_pandas()

In [ ]:
sportscode.save(code_dataset, "file.xml")

with open("file.xml", "r") as fp:
    print(fp.read())

In [ ]:
import os
os.unlink("file.xml")